In [6]:
from struct import unpack
from pathlib import Path
from enum import Enum

import json
import xmltodict
import numpy as np

import matplotlib.pyplot as plt

from astropy.time import Time
import astropy.units as u

from timdimm_tng.sequence import Sequence

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [4]:
with open("/Users/tim/timdimm.esq", 'rb') as fp:
    d = xmltodict.parse(fp)
d

{'SequenceQueue': {'@version': '2.5',
  'Observer': 'timdimm timdimm',
  'GuideDeviation': {'@enabled': 'false', '#text': '2'},
  'GuideStartDeviation': {'@enabled': 'false', '#text': '2'},
  'Autofocus': {'@enabled': 'false', '#text': '0'},
  'RefocusOnTemperatureDelta': {'@enabled': 'false', '#text': '1'},
  'RefocusEveryN': {'@enabled': 'false', '#text': '60'},
  'RefocusOnMeridianFlip': {'@enabled': 'false'},
  'Job': {'Exposure': '1',
   'Format': 'Mono',
   'Encoding': 'FITS',
   'Binning': {'X': '1', 'Y': '1'},
   'Frame': {'X': '0', 'Y': '0', 'W': '1280', 'H': '1024'},
   'Temperature': {'@force': 'false', '#text': '0'},
   'Filter': 'Red',
   'Type': 'Light',
   'Prefix': {'FilterEnabled': '0',
    'ExpEnabled': '0',
    'TimeStampEnabled': '0'},
   'Count': '1',
   'Delay': '0',
   'PreCaptureScript': '/usr/bin/true',
   'PostCaptureScript': '/usr/bin/true',
   'PreJobScript': '/usr/bin/true',
   'PostJobScript': '/usr/bin/true',
   'FITSDirectory': '/Users/tim/Pictures',
   

In [7]:
with open("timdimm_sequence_template.json", 'w') as fp:
    json.dump(d, fp, indent=4)

In [17]:
with open("../src/timdimm_tng/templates/timdimm_sequence_template.json", 'r') as fp:
    rev_d = json.load(fp)

In [18]:
with open("/Users/tim/timdimm_test.esq", 'w') as fp:
    fp.write(xmltodict.unparse(rev_d, pretty=True))